In [ ]:
!pip install pycuda

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 19.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.1/88.1 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 11.5 MB/s eta 0:00:00
  Created wheel for pycuda: filename=pycuda-2024.1-cp310-cp310-linux_x86_64.whl size=661204 sha256=4777620bbb1bb7162a1991160da0d318ceeb39eadbd585a51fc36ac9a91a7e20
  Stored in directory: /root/.cache/pip/wheels/12/34/d2/9a349255a4eca3a486d82c79d21e138ce2ccd90f414d9d72b8
Successfully built pycuda


In [ ]:
!pip install Bio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.0/281.0 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 28.0 MB/s eta 0:00:00


In [ ]:
import argparse
from Bio import SeqIO
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from scipy.sparse import coo_matrix, vstack
import pycuda.autoinit
import pycuda.driver as drv
import pycuda.gpuarray as gpuarray
from pycuda.compiler import SourceModule
import time

def read_fasta(file_path, max_length=None):
    """Lee una secuencia de un archivo FASTA y devuelve la secuencia."""
    for record in SeqIO.parse(file_path, "fasta"):
        sequence = str(record.seq)
        if max_length:
            sequence = sequence[:max_length]
        return sequence

def encode_sequence(seq):
    """Codifica una secuencia de ADN en enteros."""
    encoding = {'A': 0, 'C': 1, 'G': 2, 'T': 3}
    return np.array([encoding[nuc] for nuc in seq], dtype=np.uint8)

def generate_dotplot_cuda(seq1, seq2, window_size=1, block_size=10000):
    """Genera una matriz dispersa de dotplot para dos secuencias utilizando PyCUDA y procesamiento en ventanas."""
    len1, len2 = len(seq1), len(seq2)
    seq1_array = encode_sequence(seq1)
    seq2_array = encode_sequence(seq2)

    mod = SourceModule("""
    __global__ void dotplot_kernel(unsigned char *seq1, unsigned char *seq2, int *rows, int *cols, int len1, int len2, int window_size, int *counter) {
        int idx = blockIdx.x * blockDim.x + threadIdx.x;
        if (idx >= len1 - window_size + 1) return;

        for (int j = 0; j < len2 - window_size + 1; ++j) {
            bool match = true;
            for (int k = 0; k < window_size; ++k) {
                if (seq1[idx + k] != seq2[j + k]) {
                    match = false;
                    break;
                }
            }
            if (match) {
                int count = atomicAdd(counter, 1);
                rows[count] = idx;
                cols[count] = j;
            }
        }
    }
    """)

    total_rows, total_cols = [], []

    for start1 in range(0, len1, block_size):
        end1 = min(start1 + block_size, len1)
        seq1_block = seq1_array[start1:end1]

        for start2 in range(0, len2, block_size):
            end2 = min(start2 + block_size, len2)
            seq2_block = seq2_array[start2:end2]

            seq1_gpu = gpuarray.to_gpu(seq1_block)
            seq2_gpu = gpuarray.to_gpu(seq2_block)
            len1_gpu = np.int32(len(seq1_block))
            len2_gpu = np.int32(len(seq2_block))
            window_size_gpu = np.int32(window_size)
            counter = gpuarray.zeros(1, dtype=np.int32)
            rows_gpu = gpuarray.empty(block_size * block_size, dtype=np.int32)
            cols_gpu = gpuarray.empty(block_size * block_size, dtype=np.int32)

            grid_size = (len(seq1_block) + 256 - 1) // 256

            dotplot_kernel = mod.get_function("dotplot_kernel")
            dotplot_kernel(seq1_gpu, seq2_gpu, rows_gpu, cols_gpu, len1_gpu, len2_gpu, window_size_gpu, counter, block=(256, 1, 1), grid=(grid_size, 1, 1))

            rows = rows_gpu.get()[:counter.get()[0]]
            cols = cols_gpu.get()[:counter.get()[0]]

            total_rows.extend(rows + start1)
            total_cols.extend(cols + start2)

    dotplot = coo_matrix((np.ones(len(total_rows)), (total_rows, total_cols)), shape=(len1, len2), dtype=int)
    return dotplot.tocsr()

def plot_dotplot(dotplot, output_file):
    """Dibuja y guarda la imagen del dotplot."""
    start_time = time.time()  # Tiempo inicial para la generación de la imagen
    plt.imshow(dotplot.toarray(), cmap='Greys', interpolation='none')
    plt.savefig(output_file, format='png')
    plt.close()
    end_time = time.time()  # Tiempo final para la generación de la imagen
    print(f"Tiempo para generar y guardar la imagen: {end_time - start_time:.2f} segundos")

def main(file1, file2, output_file, max_length, block_size):
    start_time = time.time()  # Tiempo inicial para la ejecución del programa

    seq1 = read_fasta(file1, max_length)
    seq2 = read_fasta(file2, max_length)

    print(f"Longitud de la secuencia 1: {len(seq1)}")
    print(f"Longitud de la secuencia 2: {len(seq2)}")

    calc_start_time = time.time()  # Tiempo inicial para los cálculos
    dotplot = generate_dotplot_cuda(seq1, seq2, block_size=block_size)
    calc_end_time = time.time()  # Tiempo final para los cálculos

    if dotplot is not None:
        print(f"Tiempo de cálculo para generar el dotplot: {calc_end_time - calc_start_time:.2f} segundos")
        plot_dotplot(dotplot, output_file)
    else:
        print("No se pudo generar el dotplot debido a un error de memoria.")

    end_time = time.time()  # Tiempo final para la ejecución del programa
    print(f"Tiempo total de ejecución del programa: {end_time - start_time:.2f} segundos")

# Definir argumentos para el entorno Colab
class Args:
    file1 = './archivos_dotplot/E_coli.fna'
    file2 = './archivos_dotplot/Salmonella.fna'
    output = 'dotplot_cuda.png'
    max_length = 25000
    block_size = 25000

args = Args()
main(args.file1, args.file2, args.output, args.max_length, args.block_size)


Longitud de la secuencia 1: 25000
Longitud de la secuencia 2: 25000


In [ ]:
import argparse
from Bio import SeqIO
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from scipy.sparse import coo_matrix, vstack
import pycuda.autoinit
import pycuda.driver as drv
import pycuda.gpuarray as gpuarray
from pycuda.compiler import SourceModule
import time

def read_fasta(file_path, max_length=None):
    """Lee una secuencia de un archivo FASTA y devuelve la secuencia."""
    for record in SeqIO.parse(file_path, "fasta"):
        sequence = str(record.seq)
        if max_length:
            sequence = sequence[:max_length]
        return sequence

def encode_sequence(seq):
    """Codifica una secuencia de ADN en enteros."""
    encoding = {'A': 0, 'C': 1, 'G': 2, 'T': 3}
    return np.array([encoding[nuc] for nuc in seq], dtype=np.uint8)

def generate_dotplot_cuda(seq1, seq2, window_size=1, block_size=1024):
    """Genera una matriz dispersa de dotplot para dos secuencias utilizando PyCUDA."""
    len1, len2 = len(seq1), len(seq2)
    seq1_array = encode_sequence(seq1)
    seq2_array = encode_sequence(seq2)

    mod = SourceModule("""
    __global__ void dotplot_kernel(unsigned char *seq1, unsigned char *seq2, int *rows, int *cols, int len1, int len2, int window_size, int *counter) {
        int idx = blockIdx.x * blockDim.x + threadIdx.x;
        if (idx >= len1 - window_size + 1) return;

        for (int j = 0; j < len2 - window_size + 1; ++j) {
            bool match = true;
            for (int k = 0; k < window_size; ++k) {
                if (seq1[idx + k] != seq2[j + k]) {
                    match = false;
                    break;
                }
            }
            if (match) {
                int count = atomicAdd(counter, 1);
                rows[count] = idx;
                cols[count] = j;
            }
        }
    }
    """)

    total_rows, total_cols = [], []

    for start1 in tqdm(range(0, len1, block_size), desc="Processing seq1 blocks"):
        end1 = min(start1 + block_size, len1)
        seq1_block = seq1_array[start1:end1]

        for start2 in range(0, len2, block_size):
            end2 = min(start2 + block_size, len2)
            seq2_block = seq2_array[start2:end2]

            seq1_gpu = gpuarray.to_gpu(seq1_block)
            seq2_gpu = gpuarray.to_gpu(seq2_block)
            len1_gpu = np.int32(len(seq1_block))
            len2_gpu = np.int32(len(seq2_block))
            window_size_gpu = np.int32(window_size)
            counter = gpuarray.zeros(1, dtype=np.int32)
            rows_gpu = gpuarray.empty(block_size * block_size, dtype=np.int32)
            cols_gpu = gpuarray.empty(block_size * block_size, dtype=np.int32)

            grid_size = (len(seq1_block) + 255) // 256

            dotplot_kernel = mod.get_function("dotplot_kernel")
            dotplot_kernel(seq1_gpu, seq2_gpu, rows_gpu, cols_gpu, len1_gpu, len2_gpu, window_size_gpu, counter, block=(256, 1, 1), grid=(grid_size, 1, 1))

            rows = rows_gpu.get()[:counter.get()[0]]
            cols = cols_gpu.get()[:counter.get()[0]]

            total_rows.extend(rows + start1)
            total_cols.extend(cols + start2)

    dotplot = coo_matrix((np.ones(len(total_rows)), (total_rows, total_cols)), shape=(len1, len2), dtype=int)
    return dotplot.tocsr()

def plot_dotplot(dotplot, output_file):
    """Dibuja y guarda la imagen del dotplot."""
    start_time = time.time()  # Tiempo inicial para la generación de la imagen
    plt.imshow(dotplot.toarray(), cmap='Greys', interpolation='none')
    plt.savefig(output_file, format='png')
    plt.close()
    end_time = time.time()  # Tiempo final para la generación de la imagen
    print(f"Tiempo para generar y guardar la imagen: {end_time - start_time:.2f} segundos")

def main(file1, file2, output_file, max_length, block_size):
    start_time = time.time()  # Tiempo inicial para la ejecución del programa

    seq1 = read_fasta(file1, max_length)
    seq2 = read_fasta(file2, max_length)

    print(f"Longitud de la secuencia 1: {len(seq1)}")
    print(f"Longitud de la secuencia 2: {len(seq2)}")

    calc_start_time = time.time()  # Tiempo inicial para los cálculos
    dotplot = generate_dotplot_cuda(seq1, seq2, window_size=1, block_size=block_size)
    calc_end_time = time.time()  # Tiempo final para los cálculos

    if dotplot is not None:
        print(f"Tiempo de cálculo para generar el dotplot: {calc_end_time - calc_start_time:.2f} segundos")
        plot_dotplot(dotplot, output_file)
    else:
        print("No se pudo generar el dotplot debido a un error de memoria.")

    end_time = time.time()  # Tiempo final para la ejecución del programa
    print(f"Tiempo total de ejecución del programa: {end_time - start_time:.2f} segundos")

# Definir argumentos para el entorno Colab
class Args:
    file1 = './archivos_dotplot/E_coli.fna'
    file2 = './archivos_dotplot/Salmonella.fna'
    output = 'dotplot_cuda.png'
    max_length = 15000
    block_size = 15000

args = Args()
main(args.file1, args.file2, args.output, args.max_length, args.block_size)


Longitud de la secuencia 1: 15000
Longitud de la secuencia 2: 15000


Processing seq1 blocks: 100%|██████████| 1/1 [00:09<00:00,  9.15s/it]


Tiempo de cálculo para generar el dotplot: 21.36 segundos
Tiempo para generar y guardar la imagen: 5.02 segundos
Tiempo total de ejecución del programa: 26.58 segundos
